# Bring data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#bring your data!
path = input('please type your path to the database: ')
data = pd.read_csv(f'{path}')
data.head()

#/Users/danielriojas/Documents/Random_2022/clean_depressionvspossitive.csv

please type your path to the database: /Users/danielriojas/Documents/Random_2022/clean_depressionvspossitive.csv


/var/folders/qt/429xyfsn0gb9bdmc6vhhrh_h0000gn/T/ipykernel_37945/3614654796.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'{path}')


,Unnamed: 0,ids,Tweet,label
0,799999,1467822272,love health4uandpets u guys r best,0
1,800000,1467822273,im meeting one besties tonight cant wait girl ...,0
2,800001,1467822283,darealsunisakim thanks twitter add sunisa got ...,0
3,800002,1467822287,sick really cheap hurts much eat real food plu...,0
4,800003,1467822293,lovesbrooklyn2 effect everyone,0


In [3]:
positive_data = data[data.label == 0]

In [6]:
positive_data.shape

(800000, 4)

In [4]:
depression_data = data[data.label==1]

In [7]:
depression_data.shape

(2345, 4)

In [8]:
sample_positive_data = positive_data.sample( n  = 2345)

In [9]:
sample_data = sample_positive_data.append(depression_data)

/var/folders/qt/429xyfsn0gb9bdmc6vhhrh_h0000gn/T/ipykernel_37945/2366412613.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_data = sample_positive_data.append(depression_data)


In [10]:
sample_data =  sample_data.sample(frac=1)

In [11]:
sample_data.shape

(4690, 4)

In [12]:
text_col = input('choose the text column of your data: ')
X = sample_data[f'{text_col}']

target_col = input('choose the target column of your data: ')
y = sample_data[f'{target_col}']

choose the text column of your data: Tweet
choose the target column of your data: label


# naive bayes model

## cross validation

In [23]:
!pip install nltk

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 835.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 KB 651.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 KB 1.0 MB/s eta 0:00:0000:0100:01


In [25]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV , cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [26]:
tfid = TfidfVectorizer(ngram_range=(4,5))
nb = MultinomialNB()

In [27]:
X = X.apply(str)
#X.apply(word_tokenize)
X

800439    recently fallen waking going run everyday let ...
800286    miss post running mental health use running co...
173545    geofffield lol awesome must sign get lorikeets...
18433     moblie commercial quot old school quot classic...
801489                                                  nan
                                ...                        
411577                               side note paper doable
122932                               thanks jenniwalsh love
600675    going visit sister untill felt abit sick nan b...
801247    darrenestes negative better option depression ...
776548                  today matching nindynindot hahahaha
Name: Tweet, Length: 4690, dtype: object

In [28]:
vector = tfid.fit_transform(X)

In [ ]:
#X = pd.DataFrame(X.toarray(),columns = tfid.get_feature_names_out())

In [29]:
simple_cross_val_recall = cross_val_score(nb, vector , y , scoring= 'recall', cv=3,
               n_jobs = -1 , verbose = 1
               )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.3s finished


In [30]:
print('the recall: ' ,np.mean(simple_cross_val_recall))

the recall:  0.15268149234865133


In [31]:
simple_cross_val_accuracy = cross_val_score(nb, vector , y , scoring= 'accuracy', cv=3,
               n_jobs = -1 , verbose = 1
               )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.8s finished


In [32]:
print('the accuracy: ' , np.mean(simple_cross_val_accuracy) )

the accuracy:  0.551167939984695


## grid search

In [33]:
tfid2 = TfidfVectorizer(ngram_range=(4,5))
nb2 = MultinomialNB()
X = X.apply(str)
X

800439    recently fallen waking going run everyday let ...
800286    miss post running mental health use running co...
173545    geofffield lol awesome must sign get lorikeets...
18433     moblie commercial quot old school quot classic...
801489                                                  nan
                                ...                        
411577                               side note paper doable
122932                               thanks jenniwalsh love
600675    going visit sister untill felt abit sick nan b...
801247    darrenestes negative better option depression ...
776548                  today matching nindynindot hahahaha
Name: Tweet, Length: 4690, dtype: object

In [34]:
pipe = Pipeline([
    ('TfidfVectorizer', tfid2),
    ('MultinomialNB()' , nb2)
    
])

In [35]:
pipe.get_params()

{'memory': None,
 'steps': [('TfidfVectorizer', TfidfVectorizer(ngram_range=(4, 5))),
  ('MultinomialNB()', MultinomialNB())],
 'verbose': False,
 'TfidfVectorizer': TfidfVectorizer(ngram_range=(4, 5)),
 'MultinomialNB()': MultinomialNB(),
 'TfidfVectorizer__analyzer': 'word',
 'TfidfVectorizer__binary': False,
 'TfidfVectorizer__decode_error': 'strict',
 'TfidfVectorizer__dtype': numpy.float64,
 'TfidfVectorizer__encoding': 'utf-8',
 'TfidfVectorizer__input': 'content',
 'TfidfVectorizer__lowercase': True,
 'TfidfVectorizer__max_df': 1.0,
 'TfidfVectorizer__max_features': None,
 'TfidfVectorizer__min_df': 1,
 'TfidfVectorizer__ngram_range': (4, 5),
 'TfidfVectorizer__norm': 'l2',
 'TfidfVectorizer__preprocessor': None,
 'TfidfVectorizer__smooth_idf': True,
 'TfidfVectorizer__stop_words': None,
 'TfidfVectorizer__strip_accents': None,
 'TfidfVectorizer__sublinear_tf': False,
 'TfidfVectorizer__token_pattern': '(?u)\\b\\w\\w+\\b',
 'TfidfVectorizer__tokenizer': None,
 'TfidfVectorizer__

In [36]:
pipe_grid = {
    'TfidfVectorizer__ngram_range': [(1,2) , (2,3), (3,4),(4, 5)],
    'MultinomialNB()__alpha': [0.1 , 0.5 , 1.0]
    
}

In [37]:
search_recall= GridSearchCV(pipe,
    pipe_grid,
    scoring='recall',
    n_jobs=-1,
    cv=3,
    verbose=1)

In [38]:
search_recall.fit(X , y )

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TfidfVectorizer',
                                        TfidfVectorizer(ngram_range=(4, 5))),
                                       ('MultinomialNB()', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'MultinomialNB()__alpha': [0.1, 0.5, 1.0],
                         'TfidfVectorizer__ngram_range': [(1, 2), (2, 3),
                                                          (3, 4), (4, 5)]},
             scoring='recall', verbose=1)

In [39]:
print('the best params for recall: ',search_recall.best_params_)
print('the best recall score: ' , search_recall.best_score_)

the best params for recall:  {'MultinomialNB()__alpha': 1.0, 'TfidfVectorizer__ngram_range': (1, 2)}
the best recall score:  0.9744180030192782


In [40]:
best_model_recall = search_recall.best_estimator_

In [41]:
best_model_recall["MultinomialNB()"].params()

AttributeError: 'MultinomialNB' object has no attribute 'params'

In [42]:
search_accuracy= GridSearchCV(pipe,
    pipe_grid,
    scoring='accuracy',
    n_jobs=-1,
    cv=3,
    verbose=1)

In [43]:
search_accuracy.fit(X , y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TfidfVectorizer',
                                        TfidfVectorizer(ngram_range=(4, 5))),
                                       ('MultinomialNB()', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'MultinomialNB()__alpha': [0.1, 0.5, 1.0],
                         'TfidfVectorizer__ngram_range': [(1, 2), (2, 3),
                                                          (3, 4), (4, 5)]},
             scoring='accuracy', verbose=1)

In [44]:
print('the best params for accuracy: ',search_accuracy.best_params_)
print('the best accuracy score: ' , search_accuracy.best_score_)

the best params for accuracy:  {'MultinomialNB()__alpha': 0.5, 'TfidfVectorizer__ngram_range': (1, 2)}
the best accuracy score:  0.9117274799429911


# testeo y analisis del modelo

In [ ]:
'''{'MultinomialNB()__alpha': 1.0, 'TfidfVectorizer__ngram_range': (1, 2)}'''

In [45]:
tfid3 = TfidfVectorizer(ngram_range=(1,2))
nb3 = MultinomialNB(alpha = 1.0)

In [46]:
tfid3.fit(X)

TfidfVectorizer(ngram_range=(1, 2))

In [47]:
vector3 = tfid3.transform(X)

In [48]:
tfid3.get_feature_names()

/Users/danielriojas/.pyenv/versions/3.8.12/envs/MHFA/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['001',
 '001 haha',
 '01',
 '01 problem',
 '08',
 '08 14',
 '100',
 '100 flashbacks',
 '11',
 '11 status',
 '111',
 '111 middle',
 '13',
 '13 new',
 '14',
 '14 good',
 '15',
 '15 house',
 '180425',
 '180425 mscpedit',
 '1969',
 '1969 oh',
 '1988',
 '1988 well',
 '1xe',
 '1xe great',
 '22',
 '22 jessleal7',
 '23275879',
 '2d3dilumdu',
 '2d3dilumdu barbara',
 '3eb0f4cc',
 '3eb0f4cc fd6',
 '42',
 '42 sorry',
 '4562',
 '4562 src',
 '53',
 '53 depression',
 '5adf8637e4b061c0bfa27b19',
 '5adf8637e4b061c0bfa27b19 ncid',
 '6661',
 '6661 haha',
 '6cd2la93',
 '6ticzy',
 '6ticzy funniest',
 '714',
 '714 lol',
 '727',
 '727 silly',
 '76',
 '76 good',
 '76795',
 '76795 real',
 '86',
 '86 smrutimishra7',
 '99',
 '99 change',
 'a10066808',
 'a10066808 utm',
 'a1j3hy',
 'a1j3hy rekyewflst',
 'a2i',
 'a3',
 'a690a0cebade',
 'a6yvb33h2y',
 'a6zft61se',
 'a7q',
 'a8319686',
 'a8319686 html',
 'a9b8c66acf80fca040784e5e9cd',
 'aa',
 'aa mtg',
 'aa poc',
 'aaa',
 'aaa shooters',
 'aaaaaaaaaaaaaalllll',
 'a

In [49]:
nb3.fit(vector3, y)

MultinomialNB()

In [50]:
nb3.coef_

/Users/danielriojas/.pyenv/versions/3.8.12/envs/MHFA/lib/python3.8/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


array([[-11.03060179, -11.03060179, -11.03060179, ..., -10.91036042,
        -10.85249117, -10.85249117]])

In [51]:
coefs = pd.Series(nb3.coef_[0], index = tfid3.get_feature_names())


/Users/danielriojas/.pyenv/versions/3.8.12/envs/MHFA/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [52]:
coefs.sort_values()

#agregar palabras a las stopwords , por ejemplo 'twitter', 'com'
#agregar precision, accuracy, recall , f1 , etc
#min, max 
#max_features:overfitting? ~ regularizacion

#to do: traduccion?
#red de deeplearning para buscar mejor score
#testeos
#

001                  -11.030602
little android       -11.030602
litltawny benefits   -11.030602
litltawny            -11.030602
lithium right        -11.030602
                        ...    
twitter com           -7.336893
twitter               -7.331296
https                 -7.331077
com                   -7.171262
depression            -6.179418
Length: 50707, dtype: float64